In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchbearer
import Padam

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

from models import *

# Parameters setting
lr = 0.1
partial = 1/8
weight_decay = 5e-4
beta1 = 0.9
beta2 = 0.999
net = 'resnet'
betas = (beta1, beta2)

method = 'padam'

# Data preparing
print('Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)



if net == 'vggnet':
    from models import vgg
    model = vgg.VGG('VGG16', num_classes = 10)
elif net == 'resnet':
    from models import resnet
    model = resnet.ResNet18(num_classes = 10)
elif net == 'wideresnet':
    from models import wideresnet
    model = wideresnet.WResNet_cifar10(num_classes = 10, depth=16, multiplier=4)
else:
    print ('Network undefined!')
print('Using architecture: '+net)

if method == 'padam':
    optimiser = Padam.Padam(model.parameters(), lr=lr, partial = partial , weight_decay =weight_decay, betas =betas)
elif method == 'adam':
    optimiser = optim.Adam(model.parameters(),lr = lr , betas = betas,weight_decay = weight_decay, amsgrad = False)
elif method == 'amsgrad':
    optimiser = optim.Adam(model.parameters(),lr = lr , betas = betas,weight_decay = weight_decay, amsgrad = True)
elif method == 'SGD':
    optimiser = optim.SGD(model.parameters(),lr = lr, momentum=0.9, weight_decay = weight_decay)
else:
    print('Undefined optimizer')
print('Using Optimizer: '+method)    


loss_function = nn.CrossEntropyLoss()
trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy','top_5_acc']).cuda()

# Provide the data to the trial
trial.with_generators(trainloader, val_generator=testloader ,test_generator=testloader)

# Run N epochs of training
epochs=100
result1 =trial.run(epochs)

# test the performance
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

        
 
     

In [ ]:
# run this to create a csv file of loss, acc, etc.
import pandas as pd

train_loss = []
train_acc = []
test_loss = []
test_acc = []
for i in range(epochs):
    train_loss.append(result1[i]['loss'])
    train_acc.append(result1[i]['acc'])
    test_loss.append(result1[i]['val_loss'])
    test_acc.append(result1[i]['val_acc'])

df = pd.DataFrame({
    'train_loss':train_loss,
    'train_acc':train_acc,
    'test_loss':test_loss,
    'test_acc':test_acc
    
})

df.to_csv('result.csv',index = False)